In [1]:
Pkg.checkout("LowRankModels")
using LowRankModels, DataFrames, StatsBase, PyPlot

INFO: Checking out LowRankModels master...
INFO: Pulling LowRankModels latest master...
INFO: No packages to install, update or remove


In [2]:
import LowRankModels: evaluate, grad
evaluate(loss::Loss, X::Array{Float64,2}, w, y) = evaluate(loss, X*w, y)
grad(loss::Loss, X::Array{Float64,2}, w, y) = X'*grad(loss, X*w, y)


grad (generic function with 18 methods)

In [29]:
function proxgrad(loss::Loss, reg::Regularizer, X, y;
                  maxiters::Int = 10, stepsize::Number = 1., 
                  ch::ConvergenceHistory = ConvergenceHistory("proxgrad"))
    w = zeros(size(X,2))
    for t=1:maxiters
        t0 = time()
        # gradient step
        g = grad(loss, X, w, y)
        w = w - stepsize*g
        # prox step
        w = prox(reg, w, stepsize)
        # record objective value
        update_ch!(ch, time() - t0, obj = evaluate(loss, X, w, y) + evaluate(reg, w))
    end
    return w
end


function bool_convert(x::Array{Float64,1})
    len = length(x)
    y = Array{Bool}(len)
    for i in 1:len
        if x[i] == 1
            y[i] = true
        else 
            y[i] = false
            end
    end
    return y
end
    

bool_convert (generic function with 1 method)

# Model Methods

In [30]:
type Model
    target :: Symbol
    features :: Array{Symbol,1}
    #loss :: Any
    P :: Any
    loss::Loss
    reg::Regularizer
end

function get_X(model, data)
    t = model.target
    f = model.features
    X = zeros(size(data,1), length(f) - 1)
    index = 1
    for col in f
       if col != t
            X[:,index] = data[col]
            index += 1
       end
    end
    return X
end 

function get_y(model, data)
    t = model.target
    y = Array{Float64}(data[t])
    return y 
end 

function get_w(model,data)
    X = get_X(model,data) 
    y = get_y(model,data)
    L = model.loss
    R = model.reg
    if model.P == Boolean
        y = bool_convert(y)
    end
    w = proxgrad(L, R, X, y;  stepsize=1/(2*maximum(svdfact(X)[:S])^2), maxiters=50)
    return w
end

function avg_w(model,data
    ;resample_num = 1000)
    w_bs = 0
    n = size(data,1)
    for i in 1:resample_num
        #randomly samples from training set
        train_sample = data[sample(1:n,round(Int,.5*n)), :] 
        #incrememts bootstrap parameter by parameter of new sample
        w_bs = w_bs + get_w(model,data)
    end
    #finds average parameter among trials
    w_bs = w_bs / resample_num
    return w_bs
end 

function predict(model,train,test)
    w = get_w(model,train)
    X = get_X(model,test)
    #will be a function
    prediction = model.P(X,w)
    return prediction
end






predict (generic function with 1 method)

Notes: (1) Need to find a way to create a model that doesn't require regulizer (2) Boolean Loss functions don't work
(3) write a multi-class prediction function

### Prediction Space to Output Space Functions

In [5]:
function Boolean(X,w)
    val = sign(X*w)
    return val
end

function Real(X,w)
    val = X*w
    return val
end

function PosReal(X,w)
    val = max(X*w,0)
    return val
end

function Integer(X,w)
    val = round(X*w)
    return val
end

function PosInteger(X,w)
    val = max(round(X*w),0)
    return val
end


PosInteger (generic function with 1 method)

# Error/Validation Functions

In [6]:

function squared_bias(model,data;num_expectation = 1000)
    w_avg = avg_w(model,data)
    t = model.target
    f = model.features
    X = get_X(model,data)
    y = get_y(model,data)
    bias2 = 0
    for i in 1:num_expectation
        index = ceil(Int,rand()*size(X,1))
        row = (X[index,:])'
        pred_value = model.P(row,w_avg)[1]
        true_value = y[index]
        
        bias2 += ((true_value - pred_value))^2
    end
    return bias2/num_expectation
end



#need to make variance calculation faster/only reliable for real valued predictions
function variance_helper(x,model,data)
    w_avg = avg_w(model,data)
    num_samples = 100
    wMat = zeros(num_samples, length(w_avg))
    for i in 1:num_samples
        sample_data = data[sample(1:size(data,1),round(Int,.5*size(data,1))), :]
        w_dataset = get_w(model,sample_data)
        wMat[i,:] = w_dataset
    end
    return mean(((wMat .- transpose(w_avg)).^2)*x.^2)
end


function variance(model,data;num_expectation = 20)
    var = 0
    X = get_X(model,data)
    for i in 1:num_expectation
        row = X[ceil(Int,rand()*size(X,1)),:]
        var += variance_helper(row,model,data)
    end
    return var / num_expectation
end

function error_est(model,data)
    return squared_bias(model,data) + variance(model,data)
end



function bool_validate(model,train,test)
    pred_val = predict(model,train,test)
    true_val = get_y(model,test)

    n = length(pred_val)
    count = 0
    for i in 1:n
        if pred_val[i] != true_val[i]
            count +=1
        end
    end
    return (count/n)
end

function mse_validate(model,train,test)
    pred_val = predict(model,train,test)
    true_val = get_y(model,test)

    n = length(pred_val)
    
    err = mean((pred_val .- true_val).^2)
    return err
end



mse_validate (generic function with 1 method)

# Data Cleaning/Manipulation Functions

In [7]:
#Pre Condition: missing values have been removed since all entries will become column names 
function one_hot_encode(data::DataFrame, column_name::Symbol)
    n = size(data,1) #length of columns 
    binary_data = DataFrame() #new dataframe to store cleaned data
    old_row = data[column_name]
    #creates new columns out of entries of old column
    for new_col in unique(data[column_name])
        binary_data[Symbol(new_col)] = -1*ones(n)
    end
    #fills in binary values {1,-1} for each row
    for i in 1:n
        binary_data[i,Symbol(old_row[i])] = 1.0
    end
    return binary_data
end 

#Pre Condition: data is heart attack data from SPARCS dataset
function remove_missing_data(data::DataFrame)
    cdata = copy(data)
    #Length of Stay
    cdata = cdata[cdata[:Length_of_Stay] .!= "120 +",:]
    cdata[:Length_of_Stay] = float(cdata[:Length_of_Stay])
    #Payment_Typology
    cdata = cdata[cdata[:Payment_Typology_1] .!= "Miscellaneous/Other",:]
    cdata = cdata[cdata[:Payment_Typology_1] .!= "Unknown",:]
    #Patient_Dispostion 
    cdata = cdata[cdata[:Patient_Disposition] .!= "Another Type Not Listed",:]
    #Type of Admission
    cdata = cdata[cdata[:Type_of_Admission] .!= "Not Available",:]
    #Race
    cdata = cdata[cdata[:Race] .!= "Unknown",:]
    #Total Charges
    n = size(cdata,1)
    for i in 1:n
        if cdata[:Total_Charges][i][2:end] == "" #impute better
            val = "0"
        else 
            val = cdata[:Total_Charges][i][2:end]
        end
            cdata[:Total_Charges][i] = val
    end
    cdata[:Total_Charges] = float(cdata[:Total_Charges])
    return cdata
end

#Pre_Condition: data is heart_attack data from SPARCS dataset
function clean(data::DataFrame)
    temp = remove_missing_data(data)
    clean_data = DataFrame()
    categorical_col = [:Health_Service_Area,:Age_Group,:Hospital_County,
                    :Patient_Disposition,:Gender,:Race,
                    :Type_of_Admission,:Payment_Typology_1]
    for col in categorical_col
        clean_data = hcat(clean_data,one_hot_encode(temp,col))
    end
    delete!(clean_data,:F)
    clean_data[:Length_of_Stay] = temp[:Length_of_Stay]
    clean_data[:Total_Charges] = temp[:Total_Charges]
    clean_data[:APR_Severity_of_Illness_Code] = temp[:APR_Severity_of_Illness_Code]
    clean_data[:offset] = ones(size(clean_data,1))
    return clean_data
end


#helper function for split_data
function build_subset(index,data)
    subset = similar(data,length(index))
    for i in 1:length(index)
        subset[i,:] = data[index[i],:]
    end
    return subset
end 

#splits dataset into train and test sets
#splits data based on the discharge identifier column :x (integer)
#split_raio = ratio of training set size to entire dataset size
function split_data(data_set;split_ratio = .8)
    n = size(data_set,1)
    split = Int(round(split_ratio*n))
    rand_index = shuffle([i for i in 1:n])
    train_index = rand_index[1:split]
    test_index = rand_index[split + 1:n]
    train = build_subset(train_index,data_set)
    test = build_subset(test_index,data_set)
    return(train,test)
end 


split_data (generic function with 1 method)

# Testing Space

In [8]:
heart_attack = readtable("heart_attack_clean.csv")
train,test = split_data(heart_attack)


(846×90 DataFrames.DataFrame
│ Row │ Western_NY │ Finger_Lakes │ Southern_Tier │ Central_NY │
├─────┼────────────┼──────────────┼───────────────┼────────────┤
│ 1   │ 1.0        │ -1.0         │ -1.0          │ -1.0       │
│ 2   │ 1.0        │ -1.0         │ -1.0          │ -1.0       │
│ 3   │ -1.0       │ -1.0         │ -1.0          │ -1.0       │
│ 4   │ -1.0       │ -1.0         │ -1.0          │ -1.0       │
│ 5   │ -1.0       │ -1.0         │ -1.0          │ -1.0       │
│ 6   │ -1.0       │ -1.0         │ -1.0          │ -1.0       │
│ 7   │ -1.0       │ -1.0         │ -1.0          │ 1.0        │
│ 8   │ -1.0       │ -1.0         │ -1.0          │ -1.0       │
│ 9   │ -1.0       │ -1.0         │ -1.0          │ -1.0       │
│ 10  │ -1.0       │ -1.0         │ -1.0          │ -1.0       │
│ 11  │ -1.0       │ 1.0          │ -1.0          │ -1.0       │
⋮
│ 835 │ -1.0       │ -1.0         │ -1.0          │ -1.0       │
│ 836 │ -1.0       │ -1.0         │ -1.0          │ -1.0   

In [46]:
target = :Expired
f1 = [:Western_NY,:Finger_Lakes,:Southern_Tier,:Central_NY,:Capital_Adiron,:Hudson_Valley,:New_York_City,:Long_Island,
    :x70_or_Older,:x18_to_29,:x50_to_69,:x30_to_49,:x0_to_17,
   :Expired,
    :M,
    :White,:Black_African_American,:Other_Race,
    :Emergency,:Urgent,:Elective,:Trauma,:Medicare,
    :Self_Pay,:Blue_Cross_Blue_Shield,:Medicaid,:Private_Health_Insurance,:Managed_Care_Unspecified,:Federal_State_Local_VA,:Department_of_Corrections,
    :APR_Severity_of_Illness_Code,
    :offset]

m1 = Model(target,f1,Boolean, LogisticLoss(),ZeroReg())
m2 = Model(target,f1,Boolean, QuadLoss(),ZeroReg())

Model(:Expired,Symbol[:Western_NY,:Finger_Lakes,:Southern_Tier,:Central_NY,:Capital_Adiron,:Hudson_Valley,:New_York_City,:Long_Island,:x70_or_Older,:x18_to_29  …  :Medicare,:Self_Pay,:Blue_Cross_Blue_Shield,:Medicaid,:Private_Health_Insurance,:Managed_Care_Unspecified,:Federal_State_Local_VA,:Department_of_Corrections,:APR_Severity_of_Illness_Code,:offset],Boolean,LowRankModels.QuadLoss(1.0,LowRankModels.RealDomain()),LowRankModels.ZeroReg())

In [45]:
bool_validate(m1,train,test)


0.36018957345971564

In [47]:
bool_validate(m2,train,test)


0.4312796208530806